## I-JEPA Mély tanulási architektúra képi klasszifikációs feladatra

* Tiny-ImageNet adathalmazon (200 osztály) történő tanítás, tesztelés
* Előre tanított és klasszifikációra finomhangolt I-JEPA modell használata

In [11]:
# szükséges python modulok importálása
from transformers import AutoImageProcessor, IJepaForImageClassification
import torch
from datasets import load_dataset
from sklearn.metrics import classification_report

device = "cuda" if torch.cuda.is_available() else "cpu"

In [12]:
# tiny imagenet adathalmat betöltése

dataset = load_dataset("Maysee/tiny-imagenet")

In [13]:
# szükséges modellek inicializálása

# előre tanított I-JEPA modell -> csak reprezentációkat tanult
image_processor = AutoImageProcessor.from_pretrained("facebook/ijepa_vith14_1k")

# osztályozásra finomhangolt I-JEPA -> reprezentációkhoz címkéket kapcsol
model = IJepaForImageClassification.from_pretrained("facebook/ijepa_vith14_1k")

Some weights of IJepaForImageClassification were not initialized from the model checkpoint at facebook/ijepa_vith14_1k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# kiértékelő ciklus meghatározása

preds = []
labels = []

for item in dataset["valid"]:
    image = item["image"]
    label = item["label"]

    inputs = image_processor(image, return_tensors="pt").to(device)

    with torch.no_grad():
        logits = model(**inputs).logits

    pred_label = logits.argmax(-1).item()

    preds.append(pred_label)
    labels.append(label)

In [ ]:
# metrikák meghatározása (előzetes)

print(classification_report(labels, preds))